<a href="https://colab.research.google.com/github/joaochenriques/WE_LAB_2023/blob/main/DataCorrection/OWC_CamerasDataCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py
import matplotlib.pyplot as mpl
import numpy as np
import pandas as pd
import scipy.signal
from dataclasses import dataclass, field
import os, pathlib

In [2]:
if not pathlib.Path("YB2m_V6.h5").exists():
  os.system( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/DataCorrection/YB2m_V6.zip" )
  os.system( "unzip YB2m_V6.zip" )
   
if not pathlib.Path("YB4m_V6.h5").exists():
  os.system( "curl -O https://raw.githubusercontent.com/joaochenriques/WE_LAB_2023/main/DataCorrection/YB4m_V6.zip" )
  os.system( "unzip YB4m_V6.zip" )

In [3]:
os.system( "unzip YB2m_V6.zip" )

256

In [5]:
if not pathlib.Path("mpl_utils.py").exists():
  os.system( "curl -O https://raw.githubusercontent.com/joaochenriques/ipynb_libs/main/mpl_utils.py" ) 

import mpl_utils as mut
mut.config_plots()

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')

mpl.rcParams["figure.figsize"] = (6,4.5)

In [6]:
#~==============================================================================
def read_hdf_array( hdf5_Input, group, name ):
    return np.array( hdf5_Input[ group + '/' + name ] )

def read_hdf_scalar( hdf5_Input, group, name ):
    return hdf5_Input[ group + '/' + name ][()]

def read_hdf_string( hdf5_Input, group, name ):
    return hdf5_Input[ group + '/' + name ][()]

#~==============================================================================
def save_hdf_array( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata, 
                                compression="gzip", compression_opts=9  )

def save_hdf_scalar( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata )

def save_hdf_string( hdf5_Output, group, name, fdata ):
    hdf5_Output.create_dataset( group + '/' + name, data=fdata, 
                                dtype=save_hdf_string.dt_str )
save_hdf_string.dt_str = h5py.special_dtype( vlen=bytes )

In [75]:
hf = h5py.File( 'YB2m_Tests_V5.h5', 'a' )
filenames = []
cases_names = list( hf.keys() )
for case_name in cases_names:
    var_entry = case_name + '/CamerasFileName'
    if var_entry in hf:
        filename = read_hdf_string( hf, case_name, 'CamerasFileName' ).decode()

        if os.path.isfile( './' + filename ):
            print( "Found: " + filename )
            filenames.append( filename )
        else:
            print( "NOT Found: " + filename )
            # del hf[ case_name + '/CamerasFileName' ]
            # save_hdf_string( hf, case_name, 'CamerasFileName', 'YB2m_H0.02m_f0.60Hz.csv' )

hf.close()

Found: YB2m_H0.02m_f0.30Hz.csv
Found: YB2m_H0.02m_f0.50Hz.csv
Found: YB2m_H0.02m_f0.60Hz.csv
Found: YB2m_H0.02m_f0.64Hz.csv
Found: YB2m_H0.02m_f0.70Hz.csv
Found: YB2m_H0.02m_f0.74Hz.csv
Found: YB2m_H0.02m_f0.76Hz.csv
Found: YB2m_H0.02m_f0.80Hz.csv
Found: YB2m_H0.02m_f0.84Hz.csv
Found: YB2m_H0.02m_f0.88Hz.csv
Found: YB2m_H0.02m_f0.90Hz.csv
Found: YB2m_H0.02m_f0.92Hz.csv
Found: YB2m_H0.02m_f0.96Hz.csv
Found: YB2m_H0.02m_f1.00Hz.csv
Found: YB2m_H0.02m_f1.02Hz.csv
Found: YB2m_H0.02m_f1.06Hz.csv
Found: YB2m_H0.02m_f1.08Hz.csv
Found: YB2m_H0.02m_f1.10Hz.csv
Found: YB2m_H0.02m_f1.12Hz.csv
Found: YB2m_H0.02m_f1.14Hz.csv
Found: YB2m_H0.02m_f1.16Hz.csv
Found: YB2m_H0.02m_f1.18Hz.csv
Found: YB2m_H0.02m_f1.20Hz.csv
Found: YB2m_H0.02m_f1.24Hz.csv
Found: YB2m_H0.02m_f1.28Hz.csv
Found: YB2m_H0.02m_f1.30Hz.csv
Found: YB2m_H0.02m_f1.34Hz.csv
Found: YB2m_H0.02m_f1.40Hz.csv
Found: YB2m_H0.02m_f1.50Hz.csv


In [76]:
for filename in filenames:
    df = pd.read_csv( filename, skiprows=(0,1,2,3,4,5,6), header=None )
    df.keys()
    valid = True
    data = np.array(df[2])
    n = np.count_nonzero(~np.isnan(data))
    print( filename + ' -- ' + str(len(data)) + ' -- ' + str(n) )

YB2m_H0.02m_f0.30Hz.csv -- 14946 -- 2450
YB2m_H0.02m_f0.50Hz.csv -- 15078 -- 1118
YB2m_H0.02m_f0.60Hz.csv -- 15977 -- 12
YB2m_H0.02m_f0.64Hz.csv -- 16079 -- 16079
YB2m_H0.02m_f0.70Hz.csv -- 18130 -- 18130
YB2m_H0.02m_f0.74Hz.csv -- 17142 -- 17137
YB2m_H0.02m_f0.76Hz.csv -- 17295 -- 17295
YB2m_H0.02m_f0.80Hz.csv -- 15078 -- 1828
YB2m_H0.02m_f0.84Hz.csv -- 15168 -- 15168
YB2m_H0.02m_f0.88Hz.csv -- 16144 -- 16144
YB2m_H0.02m_f0.90Hz.csv -- 15268 -- 491
YB2m_H0.02m_f0.92Hz.csv -- 15827 -- 15827
YB2m_H0.02m_f0.96Hz.csv -- 16186 -- 16186
YB2m_H0.02m_f1.00Hz.csv -- 15417 -- 15414
YB2m_H0.02m_f1.02Hz.csv -- 15148 -- 15148
YB2m_H0.02m_f1.06Hz.csv -- 16198 -- 16196
YB2m_H0.02m_f1.08Hz.csv -- 17839 -- 14118
YB2m_H0.02m_f1.10Hz.csv -- 14741 -- 1438
YB2m_H0.02m_f1.12Hz.csv -- 39106 -- 8056
YB2m_H0.02m_f1.14Hz.csv -- 14809 -- 14650
YB2m_H0.02m_f1.16Hz.csv -- 15730 -- 15728
YB2m_H0.02m_f1.18Hz.csv -- 16364 -- 16263
YB2m_H0.02m_f1.20Hz.csv -- 17605 -- 2477
YB2m_H0.02m_f1.24Hz.csv -- 15164 -- 15009
YB2